In [ ]:
import pandas as pd
from pyspark import SparkContext, SparkConf

SparkContext.setSystemProperty('spark.executor.memory', '3g')
conf = SparkConf().setAppName("Process Comment").setMaster("spark://25.15.27.228:7077")
sc = SparkContext.getOrCreate(conf=conf)

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Process Comment').getOrCreate()

In [ ]:
url = 'hdfs://cris:9000/ProcessShopee/Comment/rate_sum/part-00000-2bb5d99d-0093-4c2e-bc46-0b8487d2c0db-c000.snappy.parquet'
data = spark.read.parquet(url)
data = data.toPandas()

In [ ]:
data.loc[data[data['rating_star'].astype(int) < 4].index, 'rating_star'] = 0
data.loc[data[data['rating_star'].astype(int) > 3].index, 'rating_star'] = 1
data.rating_star.value_counts()

In [ ]:
data = spark.createDataFrame(data)

In [ ]:
from pyspark.ml.feature import Word2Vec
from pyspark.sql.functions import lower, col, split

dataset = data.select(lower(col('comment')).alias('comment'), 'rating_star')
dataset = dataset.select(split(dataset.comment, ' ').alias('comment'), 'rating_star')

word2Vec = Word2Vec(vectorSize=100, seed=42, inputCol="comment", outputCol="features")
word2Vec.setMaxIter(5)
model = word2Vec.fit(dataset)

res = model.transform(dataset)
res.show()

In [ ]:
data = res.select('features', 'rating_star')
data = data.withColumnRenamed('rating_star', 'label')

In [ ]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

#split train test
(train, test) = data.randomSplit([0.6, 0.4], 1234)

layers = [100, 120, 60, 2]
# create the trainer and set its parameters
trainer = MultilayerPerceptronClassifier(maxIter=500, layers=layers, blockSize=128, seed=1234)

# train the model
model = trainer.fit(train)

In [ ]:
# compute accuracy on the test set
result = model.transform(test)
predictionAndLabels = result.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))